In [1]:
import sys
from torch_geometric.utils import dense_to_sparse, from_networkx, to_networkx
import copy
from collections import defaultdict
sys.path.append("../../")

from src.models.gcn import *
from src.models.trainable import *
from src.utils.datasets import *
from src.attacks.greedy_gd import *

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [5]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 4.468594074249268, Val Loss - 4.261735916137695, Val Accuracy - 0.116
Epoch 20, Train Loss - 0.47768130898475647, Val Loss - 1.3832261562347412, Val Accuracy - 0.71
Epoch 40, Train Loss - 0.20140530169010162, Val Loss - 1.3341807126998901, Val Accuracy - 0.729
Epoch 60, Train Loss - 0.10291904956102371, Val Loss - 1.6533875465393066, Val Accuracy - 0.716
Epoch 80, Train Loss - 0.17522713541984558, Val Loss - 1.6424568891525269, Val Accuracy - 0.735
Epoch 100, Train Loss - 0.11405123025178909, Val Loss - 2.1050448417663574, Val Accuracy - 0.72
Epoch 120, Train Loss - 0.08180216699838638, Val Loss - 2.17435622215271, Val Accuracy - 0.726
Epoch 140, Train Loss - 0.046544719487428665, Val Loss - 2.046952247619629, Val Accuracy - 0.726
Epoch 160, Train Loss - 0.07635019719600677, Val Loss - 2.295947551727295, Val Accuracy - 0.727
Epoch 180, Train Loss - 0.11360016465187073, Val Loss - 1.84902822971344, Val Accuracy - 0.755
Epoch 200, Train Loss - 0.05644615367054939, V

In [6]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.755
Initial Loss: 1.84902822971344


In [7]:
# attack
# attacked_data = from_networkx(data).to(device)
# attacked_data.x = data.x
# attacked_data.y = data.y
# attacked_data.train_mask = data.train_mask
# attacked_data.test_mask = data.test_mask
attacked_data = copy.deepcopy(data)

attacker_dirty = Metattack(attacked_data, device=device)
attacker_dirty.setup_surrogate(
        model,
        labeled_nodes=attacked_data.train_mask,
        unlabeled_nodes=attacked_data.test_mask,
        lambda_=0.0,
)
attacker_dirty.reset()
attacker_dirty.attack(0.075)#15%

Peturbing graph...:   0%|          | 0/395 [00:00<?, ?it/s]

Metattack(
  (surrogate): GCN(
    (conv): Sequential(
      (0): GCNConv()
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): GCNConv()
    )
  )
)

In [8]:
G_dirty = to_networkx(data)

In [9]:
G_dirty.number_of_edges()

10556

In [10]:
degs_dirty = defaultdict(tuple)

for k, v in attacker_dirty._added_edges.items():
    degs_dirty[v] = (k, True)

for k, v in attacker_dirty._removed_edges.items():
    degs_dirty[v] = (k, False)

for _, second in degs_dirty.items():
    u, v = second[0]
    if second[1]:
        G_dirty.add_edge(u, v)
    else:
        G_dirty.remove_edge(u, v)

In [11]:
G_dirty.number_of_edges()

10939

In [12]:
attacked_data_2 = from_networkx(G_dirty).to(device)
attacked_data_2.x = attacked_data.x
attacked_data_2.y = attacked_data.y
attacked_data_2.train_mask = attacked_data.train_mask
attacked_data_2.test_mask = attacked_data.test_mask

In [13]:
# Get initial accuracy
print("Poisoned:\n")
initial_loss, initial_accuracy = train.test(attacked_data_2)
print(f"Poisoned Accuracy: {initial_accuracy}")
print(f"Poisoned Loss: {initial_loss}")

Poisoned:

Poisoned Accuracy: 0.736
Poisoned Loss: 2.014631748199463


In [14]:
model_2 = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [15]:
model_2.reset_parameters()
train_2 = Trainable(model_2)
train_2.fit(attacked_data_2, 200)

Epoch 0, Train Loss - 3.721851348876953, Val Loss - 5.060287952423096, Val Accuracy - 0.168
Epoch 20, Train Loss - 0.359022855758667, Val Loss - 1.5024776458740234, Val Accuracy - 0.68
Epoch 40, Train Loss - 0.23323506116867065, Val Loss - 1.689511775970459, Val Accuracy - 0.679
Epoch 60, Train Loss - 0.1821385771036148, Val Loss - 1.6681135892868042, Val Accuracy - 0.693
Epoch 80, Train Loss - 0.121980682015419, Val Loss - 1.6432948112487793, Val Accuracy - 0.71
Epoch 100, Train Loss - 0.1093253567814827, Val Loss - 1.7486178874969482, Val Accuracy - 0.714
Epoch 120, Train Loss - 0.09885122627019882, Val Loss - 1.9071998596191406, Val Accuracy - 0.709
Epoch 140, Train Loss - 0.056614816188812256, Val Loss - 1.756057620048523, Val Accuracy - 0.723
Epoch 160, Train Loss - 0.08454537391662598, Val Loss - 1.8248765468597412, Val Accuracy - 0.721
Epoch 180, Train Loss - 0.08353882282972336, Val Loss - 1.9403897523880005, Val Accuracy - 0.707
Epoch 200, Train Loss - 0.06822417676448822, Val

In [16]:
# Get initial accuracy
print("Evasion:\n")
initial_loss, initial_accuracy = train_2.test(attacked_data_2)
print(f"Evasion Accuracy: {initial_accuracy}")
print(f"Evasion Loss: {initial_loss}")

Evasion:

Evasion Accuracy: 0.727
Evasion Loss: 1.9504480361938477


In [17]:
# model_save_path = "cora_gcn_model.pth"
# torch.save(model.state_dict(), model_save_path)